# 3. Interfacing with Python Libraries

In this notebook, we will explore how **Apache Spark** integrates with common Python data science libraries, specifically:

1. [pandas](https://pandas.pydata.org/)
2. [numpy](https://numpy.org/)
3. [scikit-learn](https://scikit-learn.org/)

You’ll learn:

- How to **convert Spark DataFrames** to and from pandas DataFrames.
- How to **leverage** PySpark for distributing data across your cluster.
- When (and why) you’d want to **collect** smaller datasets locally to do numpy-based manipulation.
- **Comparisons** between Spark MLlib and scikit-learn.
- Techniques for **scaling** your machine learning tasks beyond a single node.

By the end, you’ll have a robust understanding of how Spark can coexist in the typical Python data science ecosystem.

## 0. Prerequisites

This notebook assumes:

- You have a **SparkSession** available as `spark`.
- You have installed relevant Python libraries (`pandas`, `numpy`, `scikit-learn`).
- You understand basic Spark concepts like **DataFrames**, **RDDs**, **transformations**, and **actions**.

If you haven’t already, please refer to previous notebooks for setup and Spark basics.

## 1. pandas and Spark

### 1.1 Converting Spark DataFrames to pandas DataFrames

Often, you’ll want to perform local operations on a smaller subset of data that easily fits in memory on a single machine. In such scenarios, you can **collect** that data as a **pandas** DataFrame. The `toPandas()` method is the simplest approach for doing so.

> **Warning**: `toPandas()` will bring all data into the driver’s memory, so ensure the dataset isn’t too large.


In [ ]:
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.appName("InterfacingWithPythonLibs").getOrCreate()

# Example Spark DataFrame
data = [
    ("Alice", 29, "Engineer"),
    ("Bob",   35, "Doctor"),
    ("Cathy", 25, "Artist"),
    ("David", 42, "Chef")
]
columns = ["Name", "Age", "Occupation"]

spark_df = spark.createDataFrame(data, columns)
print("Spark DataFrame:")
spark_df.show()

# Convert to a pandas DataFrame (Collects all data locally!)
pdf = spark_df.toPandas()
print("\nConverted to pandas DataFrame:")
print(pdf)


The above code snippet:

- Creates a simple Spark DataFrame with 4 rows.
- Uses `.toPandas()` to bring it to the driver machine.

In real-world scenarios, you might **filter** or **sample** your Spark DataFrame first, so that only a manageable subset is converted to pandas.


### 1.2 Converting pandas DataFrames to Spark DataFrames

If you have local data in pandas (for example, from CSV files that are small enough to fit on a single machine), you can **create** a Spark DataFrame using the `spark.createDataFrame()` method.

> **Tip**: Some overhead exists when converting from pandas to Spark—Spark has to infer or confirm the schema. For large data, consider loading directly into Spark from the source instead of going through pandas.


In [ ]:
# Let's assume you already have a local pandas DataFrame 'pdf'
pdf_extra = pd.DataFrame({
    'Name': ["Evelyn", "Frank"],
    'Age': [31, 28],
    'Occupation': ["Manager", "Nurse"]
})

# Convert to Spark DataFrame
spark_df_extra = spark.createDataFrame(pdf_extra)

print("New Spark DataFrame created from pandas:")
spark_df_extra.show()


### 1.3 pandas API on Spark (pyspark.pandas)

Starting in Spark 3.2+, you can use **pandas APIs on Spark** (often referred to as **`pyspark.pandas`**). This is an **experimental** feature that attempts to let you write **pandas-like code** but execute it **distributively** on Spark. That way, you can handle bigger-than-memory data with familiar pandas syntax.

> **Note**: Performance might differ from pure Spark SQL, but for many users who love pandas, this can ease the learning curve.


In [ ]:
import pyspark.pandas as ps

# Create a pandas-on-Spark DataFrame
psdf = ps.DataFrame({
    'colA': [10, 20, 30],
    'colB': [100, 200, 300]
})
print("pandas-on-Spark DataFrame:")
print(psdf)

# You can do typical pandas operations, e.g.,
print("\npsdf.sum():")
print(psdf.sum())

Behind the scenes, `pyspark.pandas` transforms your calls into Spark transformations. This allows scaling out to large datasets that exceed normal pandas memory constraints.

## 2. numpy and Spark

### 2.1 When to Use numpy with Spark

The **numpy** library provides fast, vectorized array operations on your local machine. While Spark doesn’t distribute pure numpy arrays across the cluster, you can still use numpy in combination with Spark for:

- **Local matrix transformations** after collecting or sampling from Spark.
- **Helper functions** (e.g., random number generation, complex math) on small pieces of data.

If your data is truly massive, you typically rely on Spark transformations or Spark MLlib. But for smaller subsets or specialized numeric routines, numpy remains indispensable.


### 2.2 Example: Converting Spark Data to numpy Arrays

Below is an example of how you might take a Spark DataFrame, filter down or limit the rows, and then convert to a numpy array for custom local processing. This is common if you want to feed the data into a library that **only** accepts numpy arrays (e.g., certain specialized Python libraries without Spark integration).


In [ ]:
import numpy as np

# Suppose 'spark_df' is large. We'll create a smaller sample.
small_spark_df = spark_df.limit(2)  # just 2 rows for demonstration

# Collect to driver as a list of Row objects
rows = small_spark_df.collect()

# Convert rows to a structured numpy array, or just a list for further processing
names = [row['Name'] for row in rows]
ages = np.array([row['Age'] for row in rows])

print("Names (list):", names)
print("Ages (numpy array):", ages)

If your dataset is large, you might instead prefer distributed transformations in Spark MLlib. But for smaller tasks or specialized numeric calculations, this approach is straightforward.


## 3. scikit-learn and Spark

### 3.1 Overview

[scikit-learn](https://scikit-learn.org/) is a **single-machine** machine learning library widely used in the Python ecosystem. It offers a broad range of supervised and unsupervised algorithms (e.g., linear regression, SVM, random forest, clustering, etc.).

Spark, on the other hand, provides **Spark MLlib**—a distributed, scalable machine learning library that can handle datasets too large for a single machine.

- If your data **fits in memory** on a single machine, you might just collect it to the driver and use **scikit-learn**.
- If your data is **too large** or you need distributed training, use **Spark MLlib**.


### 3.2 Using scikit-learn Locally After Collecting Data

Below is a simple example demonstrating how you could train a **linear regression** model in scikit-learn after collecting Spark data locally. This approach is feasible only if the dataset is small enough.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Suppose we have a Spark DF with numeric columns: e.g., 'Feature1', 'Feature2', 'Label'
sample_data_for_sklearn = [
    (1.0, 10.0,  8.5),
    (2.0, 18.0, 14.0),
    (3.0, 23.0, 17.2),
    (4.0, 28.0, 20.0),
    (5.0, 39.0, 32.0)
]
schema_cols = ["Feature1", "Feature2", "Label"]
df_sklearn = spark.createDataFrame(sample_data_for_sklearn, schema_cols)

# Collect to driver in pandas
pdf_sklearn = df_sklearn.toPandas()

# Separate features and labels
X = pdf_sklearn[["Feature1", "Feature2"]]
y = pdf_sklearn["Label"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit scikit-learn model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Evaluate
score = lr_model.score(X_test, y_test)
print("R-squared on test set:", score)
print("Coefficients:", lr_model.coef_)
print("Intercept:", lr_model.intercept_)

When you run the above:

- Spark will collect all rows (5 in this example) into a pandas DataFrame.
- We use `train_test_split` and `LinearRegression` from scikit-learn to fit a simple model.

Again, this is only advisable for small to moderate datasets that your driver machine can handle in memory.


### 3.3 Spark MLlib vs. scikit-learn

Spark MLlib is designed to **scale** across multiple nodes. It uses DataFrames and distributed computing to train models on data that might be terabytes in size. scikit-learn is widely used but has a **single-machine** approach.

- **Spark MLlib** advantages:
  1. Distributable algorithms for huge datasets.
  2. Seamless integration with Spark DataFrames.
  3. Built-in support for pipelines, parameter tuning, etc.

- **scikit-learn** advantages:
  1. Massive library of algorithms and features (wider than MLlib).
  2. Simpler debugging and familiarity for many data scientists.
  3. Large ecosystem of user-contributed modules.

In practice, if your dataset can fit on one machine (or you can do partial sampling or chunk-based approaches), scikit-learn is a great choice. If you need to handle truly **big data** in parallel, Spark MLlib can help you avoid memory bottlenecks on a single node.


## 4. Additional Considerations

### 4.1 Data Serialization

When passing data between Spark and local Python processes, data must be **serialized**—converted to a format for transmission or collection. This overhead can become significant if you frequently shuttle large amounts of data back and forth. To minimize overhead, try to:

- Perform **as many transformations** as possible in Spark.
- Only **collect** subsets or aggregated results locally.
- Avoid tight loops in Python that call Spark operations repeatedly—batch them if possible.

### 4.2 UDF Performance

Some libraries (like numpy, scikit-learn) may be tempting to call in a Spark **User-Defined Function (UDF)**. However, be aware that UDFs can bypass Spark’s internal optimizations. If an equivalent function exists in `pyspark.sql.functions`, prefer the built-in Spark function.

That said, if you truly need custom logic not available in Spark, UDFs or **pandas UDFs** can be a valid approach. Just be mindful of potential performance trade-offs.


## 5. Summary

### 5.1 Key Takeaways
1. **pandas ↔ Spark**: Use `spark_df.toPandas()` and `spark.createDataFrame(pandas_df)` to move data in and out of Spark. Great for smaller datasets or local manipulations.
2. **numpy with Spark**: Perfect for specialized numeric tasks on subsets of data. Full distribution of numpy arrays is not supported, so you often rely on Spark DataFrame transformations for big data.
3. **scikit-learn vs. Spark MLlib**: scikit-learn is single-node and offers a vast algorithm library, but Spark MLlib distributes training across a cluster to handle larger datasets.
4. **Keep Data in Spark**: For large-scale pipelines, keep as much data as possible in Spark to avoid serialization overhead.

### 5.2 Next Steps

- If you plan on applying **machine learning** at scale, explore **Spark MLlib** and how it compares in **API** and **capabilities** to scikit-learn.
- Consider trying out **pyspark.pandas** for a more pandas-like experience on large datasets.
- Explore **pandas UDFs** if you need to apply vectorized computations in a distributed manner.

In the next notebook, we’ll explore **visualization** and **statsmodels** integration to see how you can generate plots and run statistical tests while still leveraging Spark.


In [ ]:
# (Optional) Stop the Spark session when you're done.
spark.stop()
print("Spark session stopped.")